In [119]:
import pandas as pd
import csv
import os
import glob

path = "data/sample/test.csv"

In [296]:
def main():
    logs_path = "data\Results"
    # Busca los archivos del directorio
    for path_file in glob.iglob(f'{logs_path}/**', recursive = True):
        # Checa si es un archivo
        if os.path.isfile(path_file) and "GRR" not in path_file:
            # Abrir el archivo csv para lectura
            with open(path_file, 'r') as logfile:
                reader = csv.reader(logfile)
                # Leer todas las filas en una lista
                rows = [row for row in reader]

                # Buscar la fila con la palabra "SN" en la fila 4 o 5
                for i, row in enumerate(rows):
                    if i in [3, 4] and "SN" in row:
                        # Comprobar si la fila ya tiene 150 ","
                        if len(row) < (len(rows[0]) + 150):
                            # Agregar 150 "," a la fila encontrada
                            row += [""] * 150
                        break

            # Abrir el archivo csv para escritura
            with open(path_file, 'w', newline='') as logfile:
                writer = csv.writer(logfile)
                # Escribir todas las filas en el archivo csv
                writer.writerows(rows)
                print(path_file)

In [126]:
path_file = path

with open(path_file, 'r') as logfile:
    reader = csv.reader(logfile)
    # Leer todas las filas en una lista
    rows = [row for row in reader]
    csv_header1 = rows[0][6:]

# Buscar la fila con la palabra "SN" en la fila 4 o 5
    for i, row in enumerate(rows):
        if i in [3, 4] and "SN" in row:
            # Copia el nombre de las pruebas en la fila para el dataframe
            row += csv_header1
            break

# Abrir el archivo csv para escritura
with open(path_file, 'w', newline='') as logfile:
    writer = csv.writer(logfile)
    # Escribir todas las filas en el archivo csv
    writer.writerows(rows)
    print(path_file)
    

data/sample/test.csv


In [150]:
# read the CSV file and store it in a DataFrame
df = pd.read_csv(path, header= 4)

# add a new column "Measurement" and initialize it with NaN values
df["Measurement"] = float("NaN")
# loop through each row of the DataFrame
for i, row in df.iterrows():
    if row["Status"] == "Failed":
        measurement = row["FirstFail"]
        # search for the measurement in the header
        measurement_column = df.columns.get_loc(measurement)
        # write the value of the measurement in the new column "Measurement"
        df.at[i, "Measurement"] = df.iloc[i, measurement_column]

# select the desired columns and store them in a new DataFrame
df = df[["SN", "Date", "CycleTime", "Status", "FirstFail", "Measurement"]]

# df.drop(df.index[1], inplace = True)

print(df.index[1])
df

1


,SN,Date,CycleTime,Status,FirstFail,Measurement
0,S224800005,23/01/2023 02:57:31 p. m.,198.66s,Failed,UUT Current Measurment,0.183029
1,S224800005,23/01/2023 03:01:49 p. m.,192.42s,Failed,1.35 VDC Measurment,1.351070
2,S224800005,23/01/2023 03:05:25 p. m.,192.60s,Passed,NaN,NaN
3,S224800005,23/01/2023 03:18:01 p. m.,198.97s,Passed,NaN,NaN
4,S224800005,23/01/2023 03:21:47 p. m.,198.96s,Failed,VDC w/7 VAC Power,1.564014
5,S224800005,23/01/2023 03:26:07 p. m.,189.78s,Failed,VDC w/18 VAC Power,23.796310


In [138]:
header_df = {'SN': [], 'Date': [], 'CycleTime': [], 'Status': [], 'FirstFail': [], 'Measurement': []}
new_df = pd.DataFrame(header_df)

# save the new DataFrame to a new CSV file
new_df.to_csv("data/sample/output.csv", index=False)
new_df

,SN,Date,CycleTime,Status,FirstFail,Measurement
